### gblinearを使ってみる！

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy as sp
import lightgbm as lgb
import category_encoders as ce
# import mojimoji
import re
from cmath import nan
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import optuna
from sklearn.metrics import mean_squared_error
import japanize_matplotlib
from sklearn.model_selection import KFold
import xgboost
from catboost import CatBoostRegressor
from catboost import Pool

###訓練データの読み込み
house_age = pd.read_csv('../data/handmade_data/house_age.csv')
area_size = pd.read_csv('../data/handmade_data/area_size.csv')
room_arrange = pd.read_csv('../data/handmade_data/room_arrange.csv')
contract_span = pd.read_csv('../data/handmade_data/contract_span.csv')#欠損値をゼロ埋めした契約期間
contract_span2 = pd.read_csv('../data/handmade_data/contract_span2.csv')#欠損値を欠損値のままにした契約期間
reg_rent = pd.read_csv('../data/handmade_data/reg_rent.csv')
floor_scores = pd.read_csv('../data/handmade_data/floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「所在階」
floor_scores2 = pd.read_csv('../data/handmade_data/floor_scores2.csv')#「所在階」と「全体の階数」が独立している「所在階」
Floor_scores = pd.read_csv('../data/handmade_data/capital_floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「全体の階数」
Floor_scores2 = pd.read_csv('../data/handmade_data/capital_floor_scores2.csv')#「所在階」と「全体の階数」が独立している「全体の階数」
stations = pd.read_csv('../data/handmade_data/stations.csv')
minits = pd.read_csv('../data/handmade_data/minits.csv')
addresses = pd.read_csv('../data/handmade_data/addresses.csv')
room_arrange = pd.read_csv('../data/handmade_data/room_arrange.csv')
buildings = pd.read_csv('../data/handmade_data/buildings.csv')
land_aveprice = pd.read_csv('../data/handmade_data/land_aveprice.csv')
land_aveprice.columns = ['平均地価']
over100m2 = pd.read_csv('../data/handmade_data/over100m2.csv')
bt75and100m2 = pd.read_csv('../data/handmade_data/bt75and100m2.csv')
bt50and75m2 = pd.read_csv('../data/handmade_data/bt50and75m2.csv')
sepa_bath_toilets = pd.read_csv('../data/handmade_data/sepa_bath_toilets.csv')
sepa_bath_toilets.columns = ['バス・トイレ別']
# washlets = pd.read_csv('../data/handmade_data/washlets.csv')
area_per_rooms = pd.read_csv('../data/handmade_data/area_per_rooms.csv')
kitchen_points = pd.read_csv('../data/handmade_data/kitchen_points.csv')
# kitchen_points.drop(['ガスコンロ'],axis=1,inplace=True)
equipments = pd.read_csv('../data/handmade_data/equipments.csv')
equipments.drop(['汲み取り','ペアガラス','プロパンガス','フローリング','下水'],axis=1, inplace=True)
bath_toilet_points = pd.read_csv('../data/handmade_data/bath_toilet_points.csv')
# bath_toilet_points.drop('追焚機能',axis=1,inplace=True)
broadcasts = pd.read_csv('../data/handmade_data/broadcasts.csv')
broadcasts.drop(['高速インターネット'],axis=1,inplace=True)
parkings = pd.read_csv('../data/handmade_data/parkings.csv')
# parkings.drop('',axis=1,inplace=True)
building = pd.read_csv('../data/handmade_data/building.csv')

rent = pd.read_csv('../data/handmade_data/rent.csv')


##テストデータの読み込み
test_house_age = pd.read_csv('../data/test_data/test_house_age.csv')
test_area_size = pd.read_csv('../data/test_data/test_area_size.csv')
test_room_arrange = pd.read_csv('../data/test_data/test_room_arrange.csv')
test_contract_span = pd.read_csv('../data/test_data/test_contract_span.csv')
test_contract_span2 = pd.read_csv('../data/test_data/test_contract_span2.csv')
test_contract_span2.columns = ['契約期間']
test_reg_rent = pd.read_csv('../data/test_data/test_reg_rent.csv')
test_floor_scores = pd.read_csv('../data/test_data/test_floor_scores.csv')
test_floor_scores2 = pd.read_csv('../data/test_data/test_floor_scores2.csv')
test_Floor_scores = pd.read_csv('../data/test_data/test_capital_floor_scores.csv')
test_Floor_scores2 = pd.read_csv('../data/test_data/test_capital_floor_scores2.csv')
test_stations = pd.read_csv('../data/test_data/test_stations.csv')
test_minits = pd.read_csv('../data/test_data/test_minits.csv')
test_addresses = pd.read_csv('../data/test_data/test_addresses.csv')
test_room_arrange = pd.read_csv('../data/test_data/test_room_arrange.csv')
test_buildings = pd.read_csv('../data/test_data/test_buildings.csv')
test_land_aveprice = pd.read_csv('../data/test_data/test_land_aveprice.csv')
test_land_aveprice.columns = ['平均地価']
test_over100m2 = pd.read_csv('../data/test_data/test_over100m2.csv')
test_bt75and100m2 = pd.read_csv('../data/test_data/test_bt75and100m2.csv')
test_bt50and75m2 = pd.read_csv('../data/test_data/test_bt50and75m2.csv')
test_sepa_bath_toilets = pd.read_csv('../data/test_data/test_sepa_bath_toilets.csv')
test_sepa_bath_toilets.columns = ['バス・トイレ別']
# test_washlets = pd.read_csv('../data/test_data/test_washlets.csv')
test_area_per_rooms = pd.read_csv('../data/test_data/test_area_per_rooms.csv')
test_kitchen_points = pd.read_csv('../data/test_data/test_kitchen_points.csv')
# test_kitchen_points.drop(['ガスコンロ'],axis=1,inplace=True)
test_equipments = pd.read_csv('../data/test_data/test_equipments.csv')
test_equipments.drop(['汲み取り','ペアガラス','プロパンガス','フローリング','下水'],axis=1, inplace=True)
test_bath_toilet_points = pd.read_csv('../data/test_data/test_bath_toilet_points.csv')
# test_bath_toilet_points.drop('追焚機能',axis=1,inplace=True)
test_broadcasts = pd.read_csv('../data/handmade_data/broadcasts.csv')
test_broadcasts.drop(['高速インターネット'],axis=1,inplace=True)
test_parkings = pd.read_csv('../data/test_data/test_parkings.csv')
# test_parkings.drop('',axis=1,inplace=True)
test_building = pd.read_csv('../data/test_data/test_building.csv')


# X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, 
#                     stations, minits, land_aveprice, addresses, room_arrange[["部屋数","L","D",'S']], 
#                     building,sepa_bath_toilets,over100m2, bt75and100m2, bt50and75m2, area_per_rooms, 
#                     ], axis=1)
# y_train = pd.read_csv('../data/handmade_data/rent.csv')
# y_train.loc[[5775], '賃料'] = y_train.loc[[5775], '賃料']/10
# X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, 
#                     test_stations, test_minits, test_land_aveprice, test_addresses, test_room_arrange[["部屋数","L","D",'S']],
#                     test_building, test_sepa_bath_toilets,test_over100m2, test_bt75and100m2, test_bt50and75m2, test_area_per_rooms,
#                     ], axis=1)
# category_lists = ['定期借家','最寄り駅', '所在地', 'L', 'D', 'S','建物構造','100m^2以上', '75m^2以上100m^2未満', '50m^2以上75m^2未満', 
#                     'バス・トイレ別',
#                     'ガスコンロ', 'コンロ2口', '給湯','システムキッチン', '独立キッチン', 'コンロ3口', 'IHコンロ', '電気コンロ',
#                     'エアコン付','シューズボックス','バルコニー','室内洗濯機置場','敷地内ごみ置き場','エレベーター','水道その他','都市ガス','タイル張','ウォークインクローゼット','2面採光','3面採光','24時間換気システム','クッションフロア','庭','床暖房','ロフト','出窓','トランクルーム','オール電化',
#                     '専用バス','バス・トイレ別','シャワー','専用トイレ','浴室乾燥機','温水洗浄便座','洗面台独立','脱衣所','追焚機能',
#                     'インターネット対応','catv','csアンテナ','bsアンテナ','光ファイバー','インターネット使用料無料',
#                     '駐車場空有','駐車場空無','駐車場無','駐車場近隣','駐輪場空有','駐輪場無','バイク置き場無','バイク置き場空有'
#                     ]

X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, land_aveprice, addresses, room_arrange[["部屋数","L","D","S"]], sepa_bath_toilets, buildings, over100m2, bt75and100m2, bt50and75m2,kitchen_points], axis=1)
y_train = pd.read_csv('../data/handmade_data/rent.csv')
y_train.loc[[5775], '賃料'] = y_train.loc[[5775], '賃料']/10
X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_land_aveprice, test_addresses, test_room_arrange[["部屋数","L","D",'S']],test_sepa_bath_toilets, test_buildings, test_over100m2, test_bt75and100m2, test_bt50and75m2,test_kitchen_points], axis=1)
category_lists = ['定期借家','最寄り駅', '所在地', 'L', 'D', 'S', 'バス・トイレ別','建物構造','100m^2以上', '75m^2以上100m^2未満', '50m^2以上75m^2未満'
'ガスコンロ', 'コンロ2口', '給湯','システムキッチン', '独立キッチン', 'コンロ3口', 'IHコンロ', '電気コンロ',]

Duplicate key in file PosixPath('/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/matplotlib/mpl-data/matplotlibrc'), line 258 ('font.family : IPAexGothic')


In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=0)
valid_rmses_list = []
valid_GBDT = np.zeros(X_train.shape[0])
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index,:]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train.loc[train_index]
    y_val = y_train.loc[valid_index]

    # 学習時に用いる検証用データ
    evals = [(X_tr, 'train'), (X_tr, 'eval')]
    evals_result = {} 
    lgb_train = lgb.Dataset(X_tr, y_tr)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)
    xgb_reg = xgboost.XGBRegressor(
        #目的関数の指定 初期値も二乗誤差です
                        # 回帰問題
                        objective='reg:linear',
                        # 学習用の指標 (RMSE)
                        eval_metric='rmse',
                        #学習のラウンド数 early_stoppingを利用するので多めに指定
                        n_estimators=50000,
                        #boosterに何を用いるか 初期値もgbtreeです
                        booster='gbtree',
                        #学習率
                        learning_rate=0.05,
                        #木の最大深さ
                        max_depth=7,
                        #シード値
                        random_state=0,
                            #正則化
                        reg_alfa=6.8594164113286965e-06,

                        sub_sample=0.16056047423047484,

                        colsample_bytree=0.6987351856651322,

                        min_child_weight=1.1103525577197584,
                        
                        evals=evals,

                        evals_result=evals_result
    )
    xgb_reg.fit(X_tr, y_tr, 
                eval_set=[(X_tr, y_tr),(X_val, y_val)], 
                early_stopping_rounds=10, 
            )
    evals_result = xgb_reg.evals_result()
    train_evals = evals_result['validation_0']['rmse']
    test_evals = evals_result['validation_1']['rmse']

    pred_y_val = xgb_reg.predict(X_val)
    rmse = mean_squared_error(y_val, pred_y_val, squared=False)
    valid_rmses_list.append(rmse)

    # 学習の課程を折れ線グラフとしてプロットする
    print('Plot metrics during training...')
    plt.figure(figsize=(10, 6))
    plt.subplot(111)
    x = range(len(train_evals))
    plt.plot(x, train_evals, label='train')
    plt.plot(x, test_evals, label='test')
    # plt.plot(x, ave_cv_evals, label='validation')
    plt.grid()
    plt.legend()
    plt.show()

valid_rmse = np.mean(valid_rmses_list)
print("whole valid rmse:", valid_rmse)

print('Start predicting...')
y_pred = xgb_reg.predict(X_test)

# feature importanceを表示
xgboost.plot_importance(xgb_reg, importance_type = "gain")
plt.show()